<a href="https://colab.research.google.com/github/annikarichardson2024/Project-2/blob/main/Discord_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install python-dotenv

In [ ]:
pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.8 MB/s eta 0:00:00


In [ ]:
pip install discord.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.7 MB/s eta 0:00:00


In [ ]:
from typing import Final
import os
from dotenv import load_dotenv
from discord import Intents, Client, Message
from random import choice, randint
import openai
import json

In [ ]:
import pprint
import requests
import requests.exceptions
import pandas as pd

In [ ]:
pip install PyNaCl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 4.7 MB/s eta 0:00:00


In [ ]:
openai.api_key = 'key'

In [ ]:
load_dotenv()
TOKEN = os.getenv('token')
print (TOKEN)


None


In [ ]:
intents = Intents.default()
intents.messages = True
client = Client(intents=intents)

In [ ]:
def ask_openai(question):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": question}
        ]
    )
    answer = response['choices'][0]['message']['content']
    return answer

In [ ]:
def get_response(user_input: str) -> str:
    lowered: str = user_input.lower()

    if lowered == '':
        return 'Well, you are quiet'
    elif 'hello' in lowered:
        return 'Hello there! You need to code your AI Bot here'
    elif 'roll dice' in lowered:
        return f'You rolled a {randint(1, 6)}'
    elif 'help' in lowered:
        return f'you can say !data, !temp, or !pressure'
    else:
        return ask_openai(user_input)

In [ ]:
#Messaging
async def send_message(message: Message,user_message: str) -> None:
    if not user_message:
        print('(Message was empty becuase intents were not enabled...prob)')
        return

#private messages response
    if is_private := user_message[0] =='?':
        user_message = user_message[1:]

    try:
        response: str= get_response(user_message)
        await message.author.send(response) if is_private else await message.channel.send(response)
    except Exception as e:
        print(e)

In [ ]:
#startup of the bot
@client.event
async def on_ready() -> None:
    print(f'{client.user} is now running!')

@client.event
async def on_message(message: Message) -> None:
    if message.author == client.user:
        return

    username: str= str(message.author)
    user_message: str = message.content
    channel: str = str(message.channel)

    print(f'[{channel}] {username}: "{user_message}"')
    await send_message(message, user_message)


In [ ]:
#Starting point

def main():
    client.run(token=TOKEN)

if __name__ == '__main__':
    main()

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
# handling basic Q and A
questions = ['Hello', 'How are you?','Goodbye', 'help']
answers = ['Hey there!','Very well.','Goodbye!', 'what is your issue?']

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
sequences_q = tokenizer.texts_to_sequences(questions)
sequences_a = tokenizer.texts_to_sequences(answers)

In [ ]:
max_length = max(max(len(seq) for seq in sequences_q), max(len(seq) for seq in sequences_a))
padded_q = tf.keras.preprocessing.sequence.pad_sequences(sequences_q, maxlen=max_length, padding='post')
padded_a = tf.keras.preprocessing.sequence.pad_sequences(sequences_a, maxlen=max_length, padding='post')

In [ ]:
embedding_dim = 2000
units = 1000
vocab_size = 5000

In [ ]:
# encoder model
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(units, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# decoder model
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# seq2seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# compiling the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [ ]:
decoder_input_data = np.zeros_like(padded_a)
decoder_input_data[:, 0] = 1

# Train  model
num_epochs = 100
model.fit([padded_q, decoder_input_data], np.expand_dims(padded_a, -1), batch_size=2, epochs=num_epochs)

In [ ]:
def preprocess(input_text):
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')
    return padded_sequence

def generate(input_sequence):
    response_sequence = np.zeros((1, max_length))
    response_sequence[0, 0] = 1
    for i in range(1, max_length):
        prediction = model.predict([input_sequence, response_sequence]).argmax(axis=2)
        response_sequence[0, i] = prediction[0, i-1]
        if prediction[0, i-1] == 2:
            break
    return response_sequence

def sequence(sequence):
    return ' '.join(tokenizer.index_word.get(i, '') for i in sequence if i > 2)

In [ ]:
def chat(input_text):
    input_sequence = preprocess(input_text)
    response_sequence = generate(input_sequence)
    response_text = sequence(response_sequence[0])
    return response_text

In [ ]:
while True: # function that adresses ending convo
    question = input('You: ')
    if question.lower() == 'i quit':
        break
    if question.lower() == 'goodbye':
        break
    if question.lower() == 'i am done':
        break
    response = chat(question)
    print(f"Chatbot: {response}")

Handling API calls

In [ ]:
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd

In [ ]:
def get_api_response(url, response_type):
    try:
        response = requests.get(url)
        response.raise_for_status()

    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    if response_type == 'json':
        result = json.dumps(response.json(), sort_keys=True, indent=4)
    elif response_type == 'dataframe':
        result = pd.json_normalize(response.json())
    else:
        result = "An unhandled error has occurred!"

    return result

In [ ]:
# this loads in an api

url = "https://restcountries.com/v3.1/name/del"
response_type = ['json', 'dataframe']
df2 = get_api_response(url, response_type[1])
df2.info()